In [1]:
import numpy as np

from heapq import nlargest

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

import spacy
from spacy.lang.en.stop_words import STOP_WORDS

from string import punctuation

## ДЗ ##

__Зробіть summary нижчевказаного тексту використовуючи бібліотеки для NLP: nltk та SpaCy__

In [2]:
text = "The Orbiter Discovery, OV-103, is considered eligible for listing in the National Register of Historic Places (NRHP) in the context of the U.S. Space Shuttle Program (1969-2011) under Criterion A in the areas of Space Exploration and Transportation and under Criterion C in the area of Engineering. Because it has achieved significance within the past fifty years, Criteria Consideration G applies. Under Criterion A, Discovery is significant as the oldest of the three extant orbiter vehicles constructed for the Space Shuttle Program (SSP), the longest running American space program to date; she was the third of five orbiters built by NASA. Unlike the Mercury, Gemini, and Apollo programs, the SSP’s emphasis was on cost effectiveness and reusability, and eventually the construction of a space station. Including her maiden voyage (launched August 30, 1984), Discovery flew to space thirty-nine times, more than any of the other four orbiters; she was also the first orbiter to fly twenty missions. She had the honor of being chosen as the Return to Flight vehicle after both the Challenger and Columbia accidents. Discovery was the first shuttle to fly with the redesigned SRBs, a result of the Challenger accident, and the first shuttle to fly with the Phase II and Block I SSME. Discovery also carried the Hubble Space Telescope to orbit and performed two of the five servicing missions to the observatory. She flew the first and last dedicated Department of Defense (DoD) missions, as well as the first unclassified defense-related mission. In addition, Discovery was vital to the construction of the International Space Station (ISS); she flew thirteen of the thirty-seven total missions flown to the station by a U.S. Space Shuttle. She was the first orbiter to dock to the ISS, and the first to perform an exchange of a resident crew. Under Criterion C, Discovery is significant as a feat of engineering. According to Wayne Hale, a flight director from Johnson Space Center, the Space Shuttle orbiter represents a “huge technological leap from expendable rockets and capsules to a reusable, winged, hypersonic, cargo-carrying spacecraft.” Although her base structure followed a conventional aircraft design, she used advanced materials that both minimized her weight for cargo-carrying purposes and featured low thermal expansion ratios, which provided a stable base for her Thermal Protection System (TPS) materials. The Space Shuttle orbiter also featured the first reusable TPS; all previous spaceflight vehicles had a single-use, ablative heat shield. Other notable engineering achievements of the orbiter included the first reusable orbital propulsion system, and the first two-fault-tolerant Integrated Avionics System. As Hale stated, the Space Shuttle remains “the largest, fastest, winged hypersonic aircraft in history,” having regularly flown at twenty-five times the speed of sound."

### __Декілька функцій__ ###

In [3]:
def freq_of_word(words:list) -> dict:
    freq_of_word={}
    for word in words:
        freq_of_word[word] = freq_of_word.get(word, 0) + 1

    max_freq=max(freq_of_word.values())

    return {key: value/max_freq for key, value in freq_of_word.items()}  


def sent_scores(sent_tokens: list, freq_of_word: dict, core: str) -> dict:
    sent_scores = {}
    for sent in sent_tokens:
        if core == "ntlk":
            for word in word_tokenize(sent):
                word = word.lower()
                if word in freq_of_word.keys():
                    sent_scores[sent] = sent_scores.get(sent, 0) + freq_of_word[word] 
        elif core == "spacy":  
            key_str = str(sent)
            for word in sent:
                word = word.text.lower()
                if word in freq_of_word.keys():
                    sent_scores[key_str] = sent_scores.get(key_str, 0) + freq_of_word[word]            
    return sent_scores  


def calc_summary(sent_scores: dict, percentage: int = 30):
    len_summary = int(len(sent_scores.values()) * percentage / 100.)
    summary = nlargest(len_summary, sent_scores, key=sent_scores.get)

    final_summary = [sent for sent in summary]
    summary = " ".join(final_summary)
    
    return summary

### __Роботу буду виконувати паралельно для обох бібліотек, порівнюючи результати.__ ###

__Завантаження необхідних данних__

In [4]:
nltk.download("stopwords", quiet=True)
nlp = spacy.load("en_core_web_lg")

__Формування списків стоп-слів__

In [5]:
ignored_tokens_ntlk = stopwords.words("english")
ignored_tokens_spacy = list(STOP_WORDS)

print(len(ignored_tokens_ntlk), len(ignored_tokens_spacy))

179 326


Список для spacy майже в 2 рази більший.

__Кількість загальних стоп-слів__

In [6]:
common_tokens = set(ignored_tokens_spacy) & set(ignored_tokens_ntlk)
len(common_tokens)

123

__Що ж такого є в списку spacy та немає в ntlk__

In [7]:
np.array([token for token in ignored_tokens_spacy if token not in ignored_tokens_ntlk])

array(['latterly', 'often', 'made', 'though', 'still', 'due',
       'whereafter', 'sometimes', '‘d', 'herein', 'part', 'although',
       'behind', 'throughout', 'third', 'moreover', 'anyone', 'except',
       'say', 'becoming', 'seems', 'one', 'beyond', 'quite', "'d", "'ll",
       'namely', 'must', 'six', 'nine', 'anywhere', 'somewhere',
       'seeming', 'anyway', 'mostly', 'otherwise', '’ll', 'neither',
       'whoever', 'indeed', 'thru', "'m", 'hence', 'whenever', 'three',
       'show', 'twelve', 'formerly', 'ever', '‘ve', 'along', 'two',
       'whereby', 'seemed', 'afterwards', 'former', 'without', 'onto',
       'rather', 'last', 'whose', 'yet', '’s', '‘re', 'take', 'hereafter',
       'toward', 'alone', 'n’t', 'become', 'front', 'became', 'least',
       'five', 'eight', 'whither', 'perhaps', 'n‘t', 'whence', 'twenty',
       'beforehand', 'sixty', '’d', 'whatever', 'mine', 'move', 'within',
       'seem', 'others', 'forty', 'would', 'thus', 'various', '‘s',
       'everythi

In [8]:
# ну і навпаки
np.array([token for token in ignored_tokens_ntlk if token not in ignored_tokens_spacy])

array(["you're", "you've", "you'll", "you'd", "she's", "it's", 'theirs',
       "that'll", 'having', 's', 't', 'don', "don't", "should've", 'd',
       'll', 'm', 'o', 've', 'y', 'ain', 'aren', "aren't", 'couldn',
       "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't",
       'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma',
       'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't",
       'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't",
       'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"],
      dtype='<U9')

Хто добре знає англійську скаже чи це добре чи ні. Я вивчав та вже забув німецьку.

__Скомпоную списки стоп-слів з пунктуацією__

In [9]:
ignored_tokens_ntlk += list(punctuation + '\n')
ignored_tokens_spacy += list(punctuation + '\n')

In [10]:
np.array(ignored_tokens_ntlk)[-50:]

array(["mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan',
       "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren',
       "weren't", 'won', "won't", 'wouldn', "wouldn't", '!', '"', '#',
       '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':',
       ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{',
       '|', '}', '~', '\n'], dtype='<U10')

__Токенізація з очищенням від стоп-слів та пунктуації__

In [11]:
tokens_ntlk = word_tokenize(text)
words_ntlk = [token.lower() for token in tokens_ntlk if token.lower() not in ignored_tokens_ntlk]

doc = nlp(text)
words_spacy = [token.text.lower() for token in doc if token.text.lower() not in ignored_tokens_spacy]
print(len(words_ntlk), len(words_spacy))

279 259


__Порахуємо вагу кожного токену пропорційно частоти появи його в тексті__

In [12]:
freq_of_word_ntlk = freq_of_word(words_ntlk)
freq_of_word_ntlk_sorted = dict(sorted(freq_of_word_ntlk.items(), key=lambda x: x[1], reverse=True))

[(key, value) for key, value in freq_of_word_ntlk_sorted.items()][:10]

[('space', 1.0),
 ('first', 0.7692307692307693),
 ('shuttle', 0.6153846153846154),
 ('orbiter', 0.5384615384615384),
 ('discovery', 0.5384615384615384),
 ('criterion', 0.3076923076923077),
 ('missions', 0.3076923076923077),
 ('program', 0.23076923076923078),
 ('engineering', 0.23076923076923078),
 ('station', 0.23076923076923078)]

In [14]:
freq_of_word_spacy = freq_of_word(words_spacy)
freq_of_word_spacy_sorted = dict(sorted(freq_of_word_spacy.items(), key=lambda x: x[1], reverse=True))

[(key, value) for key, value in freq_of_word_spacy_sorted.items()][:10]

[('space', 1.0),
 ('shuttle', 0.6153846153846154),
 ('orbiter', 0.5384615384615384),
 ('discovery', 0.5384615384615384),
 ('criterion', 0.3076923076923077),
 ('missions', 0.3076923076923077),
 ('program', 0.23076923076923078),
 ('engineering', 0.23076923076923078),
 ('station', 0.23076923076923078),
 ('flew', 0.23076923076923078)]

__Розіб'ємо текст на речення__

In [15]:
sent_tokens_ntlk = sent_tokenize(text)
sent_tokens_spacy = list(doc.sents)
print(len(sent_tokens_ntlk), len(sent_tokens_spacy))

16 17


Бібліотеки відпрацювали по різному і це скажеться на кінцевому результату.

In [16]:
dif_sent_1 = [sent for sent in sent_tokens_spacy if str(sent) not in sent_tokens_ntlk]
for sent in dif_sent_1:
    print(sent)
    print(100*'-')
print(f"Number of sentences: {len(dif_sent_1)}")

According to Wayne Hale, a flight director from Johnson Space Center, the Space Shuttle orbiter represents a “huge technological leap from expendable rockets and capsules to a reusable, winged, hypersonic, cargo-carrying spacecraft.”
----------------------------------------------------------------------------------------------------
Although her base structure followed a conventional aircraft design, she used advanced materials that both minimized her weight for cargo-carrying purposes and featured low thermal expansion ratios, which provided a stable base for her Thermal Protection System (TPS) materials.
----------------------------------------------------------------------------------------------------
Number of sentences: 2


In [17]:
dif_sent_2 = [sent for sent in sent_tokens_ntlk if sent not in [str(s) for s in sent_tokens_spacy]]
for sent in dif_sent_2:
    print(sent)
    print(100*'-')
print(f"Number of sentences: {len(dif_sent_2)}")

According to Wayne Hale, a flight director from Johnson Space Center, the Space Shuttle orbiter represents a “huge technological leap from expendable rockets and capsules to a reusable, winged, hypersonic, cargo-carrying spacecraft.” Although her base structure followed a conventional aircraft design, she used advanced materials that both minimized her weight for cargo-carrying purposes and featured low thermal expansion ratios, which provided a stable base for her Thermal Protection System (TPS) materials.
----------------------------------------------------------------------------------------------------
Number of sentences: 1


Ntlk не змогла розділити вищенаведене речення на 2 і воно звичайно матиме більшу вагу при розрахунку summary

__Розрахунок вагових коефіцієнтів для речень__

In [18]:
sent_scores_ntlk = sent_scores(sent_tokens_ntlk, freq_of_word_ntlk, "ntlk")
sent_scores_ntlk_sorted = dict(sorted(sent_scores_ntlk.items(), key=lambda x: x[1], reverse=True))
sent_scores_ntlk_sorted

{'According to Wayne Hale, a flight director from Johnson Space Center, the Space Shuttle orbiter represents a “huge technological leap from expendable rockets and capsules to a reusable, winged, hypersonic, cargo-carrying spacecraft.” Although her base structure followed a conventional aircraft design, she used advanced materials that both minimized her weight for cargo-carrying purposes and featured low thermal expansion ratios, which provided a stable base for her Thermal Protection System (TPS) materials.': 8.461538461538458,
 'The Orbiter Discovery, OV-103, is considered eligible for listing in the National Register of Historic Places (NRHP) in the context of the U.S. Space Shuttle Program (1969-2011) under Criterion A in the areas of Space Exploration and Transportation and under Criterion C in the area of Engineering.': 6.23076923076923,
 'Under Criterion A, Discovery is significant as the oldest of the three extant orbiter vehicles constructed for the Space Shuttle Program (SSP

In [19]:
sent_scores_spacy = sent_scores(sent_tokens_spacy, freq_of_word_spacy, "spacy")
sent_scores_spacy_sorted = dict(sorted(sent_scores_spacy.items(), key=lambda x: x[1], reverse=True))
sent_scores_spacy_sorted

{'The Orbiter Discovery, OV-103, is considered eligible for listing in the National Register of Historic Places (NRHP) in the context of the U.S. Space Shuttle Program (1969-2011) under Criterion A in the areas of Space Exploration and Transportation and under Criterion C in the area of Engineering.': 6.307692307692307,
 'Under Criterion A, Discovery is significant as the oldest of the three extant orbiter vehicles constructed for the Space Shuttle Program (SSP), the longest running American space program to date; she was the third of five orbiters built by NASA.': 5.769230769230768,
 'According to Wayne Hale, a flight director from Johnson Space Center, the Space Shuttle orbiter represents a “huge technological leap from expendable rockets and capsules to a reusable, winged, hypersonic, cargo-carrying spacecraft.”': 5.615384615384616,
 'In addition, Discovery was vital to the construction of the International Space Station (ISS); she flew thirteen of the thirty-seven total missions fl

__Розрахунок summary__

__SPACY__

In [20]:
print(calc_summary(sent_scores_spacy, 25))

The Orbiter Discovery, OV-103, is considered eligible for listing in the National Register of Historic Places (NRHP) in the context of the U.S. Space Shuttle Program (1969-2011) under Criterion A in the areas of Space Exploration and Transportation and under Criterion C in the area of Engineering. Under Criterion A, Discovery is significant as the oldest of the three extant orbiter vehicles constructed for the Space Shuttle Program (SSP), the longest running American space program to date; she was the third of five orbiters built by NASA. According to Wayne Hale, a flight director from Johnson Space Center, the Space Shuttle orbiter represents a “huge technological leap from expendable rockets and capsules to a reusable, winged, hypersonic, cargo-carrying spacecraft.” In addition, Discovery was vital to the construction of the International Space Station (ISS); she flew thirteen of the thirty-seven total missions flown to the station by a U.S. Space Shuttle.


__NTLK__

In [21]:
print(calc_summary(sent_scores_ntlk, 25))

According to Wayne Hale, a flight director from Johnson Space Center, the Space Shuttle orbiter represents a “huge technological leap from expendable rockets and capsules to a reusable, winged, hypersonic, cargo-carrying spacecraft.” Although her base structure followed a conventional aircraft design, she used advanced materials that both minimized her weight for cargo-carrying purposes and featured low thermal expansion ratios, which provided a stable base for her Thermal Protection System (TPS) materials. The Orbiter Discovery, OV-103, is considered eligible for listing in the National Register of Historic Places (NRHP) in the context of the U.S. Space Shuttle Program (1969-2011) under Criterion A in the areas of Space Exploration and Transportation and under Criterion C in the area of Engineering. Under Criterion A, Discovery is significant as the oldest of the three extant orbiter vehicles constructed for the Space Shuttle Program (SSP), the longest running American space program t

## __Висновок__ ##

- обидві бібліотеки працюють дуже швидко, але мають, я сподіваюсь, некритичні відмінності в токенізації та виокремленні речень, що відображається на кінцевому результаті;
- обидві бібліотеки можуть бути застосовані як базові для побудування більш складних систем, наприклад для анотації текстів;
- препроцесінг тексту відіграє важливу роль та впливає на кінцевий результат.